In [204]:
import pandas as pd
import numpy as np
import matplotlib as mlt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [205]:
df=pd.read_csv("C:/Users/dell/OneDrive/Desktop/train.csv")


In [206]:
pd.set_option('display.max_columns',None)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [207]:
df.shape

(1460, 81)

change all columns_name into lowercase

In [208]:
df.columns=df.columns.str.lower()

drop unnecessary columns

In [209]:
df.drop(['id'],axis=1,inplace=True)

split columns into categories_columns and numerical_columns

In [210]:
num_cols=list(set(df.describe().columns))
obj_cols=list(set(df.columns)-set(num_cols))

check Null values in num_cols

In [211]:
nulls=df[num_cols].isnull().sum().reset_index().rename(columns={0:"Freq_null"}).sort_values(by="Freq_null",ascending=False)
nulls['Percentage']=nulls['Freq_null']/1460*100
nulls[nulls['Freq_null']>0]

,index,Freq_null,Percentage
21,lotfrontage,259,17.739726
34,garageyrblt,81,5.547945
26,masvnrarea,8,0.547945


Handle nulls

In [212]:
# Divide columns based on the level of nulls to determine the appropriate action for each.
drop_rows=["masvnrarea"]
fill_columns=["lotfrontage","garageyrblt"]

In [213]:
df.dropna(subset=drop_rows,axis=0,inplace=True)

In [214]:
df[fill_columns].boxplot()

<Axes: >

In [215]:
df['lotfrontage'].fillna(df['lotfrontage'].median(),inplace=True) # bec. there are outliers
df['garageyrblt'].fillna(df['garageyrblt'].mean(),inplace=True)

Check null values in obj_cols

In [216]:
# function to drop all columns have null_values >= 80%
threshold=0.8
x=df[obj_cols].isnull().sum().reset_index().rename(columns={0:"frequency","index":"columns"})
x["percentage"]=x["frequency"]/df.shape[0]
drops=x["columns"][x["percentage"]>=threshold]
df.drop(columns=drops,inplace=True)

In [217]:
num_cols=list(set(df.describe().columns))
obj_cols=list(set(df.columns)-set(num_cols))

Fill nulls

In [218]:
# function to fill all categorical columns with mode
def fill_null(obj_columns):
   x=df[obj_columns].isnull().sum().reset_index().rename(columns={0:"frequency","index":"columns"})
   fills=x["columns"][x["frequency"]>0]
   df[fills].fillna(df[fills].mode(),inplace=True)

In [219]:
fill_null(obj_cols)

dropping any column has 80% of the same value

In [220]:
def check_unique(col,thre):
    freq=df[col].value_counts().reset_index()
    for i in freq['count']:
        per=i/df[col].shape[0]
        if per >= thre:
            df.drop(col,axis=1,inplace=True)
            print(col)
            break

In [221]:
for i in df.columns:
    check_unique(i,0.8)

street
landcontour
utilities
landslope
condition1
condition2
bldgtype
roofmatl
extercond
bsmtcond
bsmtfintype2
bsmtfinsf2
heating
centralair
electrical
lowqualfinsf
bsmthalfbath
kitchenabvgr
functional
garagequal
garagecond
paveddrive
enclosedporch
3ssnporch
screenporch
poolarea
miscval
saletype
salecondition


update columns name

In [222]:
num_cols=list(set(df.describe().columns))
obj_cols=list(set(df.columns)-set(num_cols))

drop columns have high correlation with other column and low correlation with target

In [223]:
num_cols_without_target=list(set(num_cols)-set('saleprice'))
corr_matrix=df[num_cols_without_target].corr()
sz=corr_matrix.shape[1]

for i in range(sz):
  for j in range(i+1,sz):

    if np.abs(corr_matrix.values[i,j]) > 0.7:
      x=abs(np.corrcoef(df['saleprice'],df[num_cols_without_target[i]])[0,1])
      y=abs(np.corrcoef(df['saleprice'],df[num_cols_without_target[j]])[0,1])
      if(x<y):
         
         df.drop([num_cols_without_target[i]],axis=1,inplace=True)
         print(num_cols_without_target[i])

      else:
        
        df.drop([num_cols_without_target[j]],axis=1,inplace=True)
        print(num_cols_without_target[j])

garagearea
1stflrsf
totrmsabvgrd
grlivarea
overallqual
garageyrblt


update columns name

In [224]:
num_cols=list(set(df.describe().columns))
obj_cols=list(set(df.columns)-set(num_cols))

clamp otliers

In [225]:
for col in num_cols:
  Q1=df[col].quantile(0.25)
  Q3=df[col].quantile(0.75)
  IQR=Q3-Q1
  lower=Q1-(1.5*IQR)
  upper=Q3+(1.5*IQR)
  df[col][df[col]>upper]=upper
  df[col][df[col]<lower]=lower

check skweness of data

In [226]:
def calculate_skweness(col):
   avg=df[col].mean()
   n=df[col].shape[0]
   sum2=0
   sum3=0
   for i in df[col]:
    sum2=sum2+(i-avg)**2
    sum3=sum3+(i-avg)**3

   Q=np.sqrt(sum2/n)
   skwe=(sum3)/((n-1)*(Q**3))
   return skwe

In [227]:
for col_name in num_cols:
   ans=calculate_skweness(col_name)
   print(col_name,ans)

garagecars -0.3871261843745724
totalbsmtsf 0.2283684152707831
2ndflrsf 0.8012058575841032
mssubclass 1.0186992117843652
bsmtfinsf1 0.7438539941122371
bsmtunfsf 0.7972124739544213
yearremodadd -0.49710954707519234
openporchsf 1.1427169773431118
mosold 0.21013560164210573
bsmtfullbath 0.5719013740261712
overallcond 0.5783553838875072
lotfrontage 0.09438426281702583
saleprice 0.8072839056627888
fireplaces 0.5827189418301743
yrsold 0.09584457249200894
masvnrarea 1.2901089364548781
bedroomabvgr -0.25099083810427514
yearbuilt -0.5891841444832834
halfbath 0.6827953148042425
wooddecksf 1.0827393287745946
lotarea 0.2090910220619546
fullbath 0.03643383645290766


handle skweness by log

In [228]:
def handle_skweness_by_log(num_cols):
  for col in num_cols:
    sk=calculate_skweness(col)
    if(abs(sk)>1) :
      print(col)
      for i in df[col]:
        df[col][i]=np.log(1+abs(i))

In [229]:
handle_skweness_by_log(num_cols)

mssubclass
openporchsf
masvnrarea
wooddecksf


checking for each feature if it follow normal distribution or not using Shapiro wilk test

In [230]:
from scipy.stats import shapiro 

alpha=0.95
fit_std_scale=[]
fit_min_max_scale=[]

for i in num_cols:
    w,p=shapiro(df[i])
    print(i,w,p)
    if w > alpha:
       fit_std_scale.append(i)
    else:
        fit_min_max_scale.append(i)


garagecars 0.8336703181266785 2.014001935816895e-36
totalbsmtsf 0.9756856560707092 5.99997694011715e-15
2ndflrsf 0.7668527364730835 3.118169342815583e-41
mssubclass 0.8386193513870239 5.237942990698648e-36
bsmtfinsf1 0.8851626515388489 1.6717811303227874e-31
bsmtunfsf 0.9319015145301819 3.362134299256169e-25
yearremodadd 0.8635327816009521 9.51166957837697e-34
openporchsf 0.7636666893959045 1.9681236931442056e-41
mosold 0.9688436985015869 3.739136948403463e-17
bsmtfullbath 0.6579182147979736 0.0
overallcond 0.8190979957580566 1.370447589438119e-37
lotfrontage 0.9664708375930786 7.72704978952123e-18
saleprice 0.9372479319572449 2.7905000000497975e-24
fireplaces 0.7542260885238647 5.1834030195374983e-42
yrsold 0.8968685269355774 3.808782252596498e-30
masvnrarea 0.6386470794677734 0.0
bedroomabvgr 0.8520214557647705 7.884645577296023e-35
yearbuilt 0.9263235330581665 4.2119135703984343e-26
halfbath 0.6375049948692322 0.0
wooddecksf 0.7253619432449341 1.1210387714598537e-43
lotarea 0.978557

In [231]:
print("follow normal distribution",fit_std_scale)
print("Not follow normal distribution",fit_min_max_scale)

follow normal distribution ['totalbsmtsf', 'mosold', 'lotfrontage', 'lotarea']
Not follow normal distribution ['garagecars', '2ndflrsf', 'mssubclass', 'bsmtfinsf1', 'bsmtunfsf', 'yearremodadd', 'openporchsf', 'bsmtfullbath', 'overallcond', 'saleprice', 'fireplaces', 'yrsold', 'masvnrarea', 'bedroomabvgr', 'yearbuilt', 'halfbath', 'wooddecksf', 'fullbath']


MIN/MAX Scaling & standardizing NUmerical Features

MIN/MAX scaling

In [232]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
min_max_scaler_model=minmax_scaler.fit(df[fit_min_max_scale])
min_max_scaled_data=pd.DataFrame( min_max_scaler_model.transform(df[fit_min_max_scale]), columns=fit_min_max_scale)

standardizing

In [233]:
from sklearn.preprocessing import StandardScaler


std_scaler = StandardScaler()
std_scaler_model=std_scaler.fit(df[fit_std_scale])
std_scaled_data=pd.DataFrame( std_scaler_model.transform(df[fit_std_scale]), columns=fit_std_scale)

merge numerical feature

In [234]:
numerical_df=std_scaled_data.merge( min_max_scaled_data, right_index=True, left_index=True)

split categorical features into ordinal or nominal

In [235]:
obj_cols

['lotconfig',
 'garagefinish',
 'neighborhood',
 'mszoning',
 'garagetype',
 'kitchenqual',
 'foundation',
 'housestyle',
 'exterior2nd',
 'exterqual',
 'roofstyle',
 'masvnrtype',
 'exterior1st',
 'heatingqc',
 'fireplacequ',
 'lotshape',
 'bsmtqual',
 'bsmtexposure',
 'bsmtfintype1']

In [236]:
ordinal_cols=['bsmtexposure','bsmtqual', 'fireplacequ','mszoning','exterqual','heatingqc','bsmtfintype1','garagefinish','lotshape','kitchenqual']
nominal_cols=['foundation','roofstyle', 'housestyle','exterior2nd','masvnrtype','lotconfig','exterior1st','neighborhood','garagetype']

LabelEncoding

In [237]:
from sklearn.preprocessing import LabelEncoder

label=LabelEncoder()
label_encoded_data=pd.DataFrame()
for i in ordinal_cols:
   label_model=label.fit(df[i])
   label_encoded_data[i]=label_model.transform(df[i])

OnehotEncoding

In [238]:
from sklearn.preprocessing import OneHotEncoder

onehot=OneHotEncoder(sparse=False)
onehot_model=onehot.fit(df[nominal_cols])
onehot_encoded_data=onehot_model.transform(df[nominal_cols])

a=onehot_model.categories_
col_names=[]
for list_ in a:
    for element in list_:
        col_names.append(element)

onehot_df=pd.DataFrame(onehot_encoded_data,columns=col_names)

merge categorical columns

In [239]:
categorical_df=label_encoded_data.merge(onehot_df, right_index=True, left_index=True)

Merge numerical_dataframe and categorical_dataframe

In [240]:
final_data=categorical_df.merge(numerical_df, right_index=True, left_index=True, how='inner')

In [241]:
final_data

,bsmtexposure,bsmtqual,fireplacequ,mszoning,exterqual,heatingqc,bsmtfintype1,garagefinish,lotshape,kitchenqual,BrkTil,CBlock,PConc,Slab,Stone,Wood,Flat,Gable,Gambrel,Hip,Mansard,Shed,1.5Fin,1.5Unf,1Story,2.5Fin,2.5Unf,2Story,SFoyer,SLvl,AsbShng,AsphShn,Brk Cmn,BrkFace,CBlock,CmentBd,HdBoard,ImStucc,MetalSd,Other,Plywood,Stone,Stucco,VinylSd,Wd Sdng,Wd Shng,BrkCmn,BrkFace,Stone,NaN,Corner,CulDSac,FR2,FR3,Inside,AsbShng,AsphShn,BrkComm,BrkFace,CBlock,CemntBd,HdBoard,ImStucc,MetalSd,Plywood,Stone,Stucco,VinylSd,Wd Sdng,WdShing,Blmngtn,Blueste,BrDale,BrkSide,ClearCr,CollgCr,Crawfor,Edwards,Gilbert,IDOTRR,MeadowV,Mitchel,NAmes,NPkVill,NWAmes,NoRidge,NridgHt,OldTown,SWISU,Sawyer,SawyerW,Somerst,StoneBr,Timber,Veenker,2Types,Attchd,Basment,BuiltIn,CarPort,Detchd,NaN,totalbsmtsf,mosold,lotfrontage,lotarea,garagecars,2ndflrsf,mssubclass,bsmtfinsf1,bsmtunfsf,yearremodadd,openporchsf,bsmtfullbath,overallcond,saleprice,fireplaces,yrsold,masvnrarea,bedroomabvgr,yearbuilt,halfbath,wooddecksf,fullbath
0,3,2,5,3,2,0,2,1,3,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.485213,-1.600089,-0.236547,-0.332331,0.571429,0.469231,0.401221,0.399717,0.088955,0.883333,0.000000,0.4,0.375,0.568714,0.0,0.50,0.000000,0.625,0.944,0.5,0.000000,0.666667
1,1,2,4,3,3,0,0,1,3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.537571,-0.488448,0.635194,-0.012295,0.571429,0.000000,0.119442,0.553715,0.168421,0.433333,0.000000,0.0,1.000,0.480262,0.4,0.25,0.001670,0.625,0.728,0.0,0.709524,0.666667
2,2,2,4,3,2,0,2,1,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.323986,0.993739,-0.062199,0.446886,0.571429,0.475824,0.401221,0.275159,0.257376,0.866667,0.247059,0.4,0.375,0.617854,0.4,0.50,0.390361,0.625,0.928,0.5,0.000000,0.666667
3,3,3,2,3,3,2,0,2,0,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.737130,-1.600089,-0.527127,-0.026210,0.857143,0.415385,0.471665,0.122293,0.320237,0.333333,0.205882,0.4,0.375,0.344308,0.4,0.00,0.000000,0.625,0.240,0.0,0.000000,0.333333
4,0,2,4,3,2,0,2,1,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.242828,2.105380,0.867659,1.284544,0.857143,0.578571,0.401221,0.370842,0.290586,0.833333,0.009467,0.4,0.375,0.704668,0.4,0.50,0.843373,0.875,0.920,0.5,0.457143,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

split data into x,y

In [242]:
x=final_data.drop(['saleprice'],axis=1)
y=final_data['saleprice']

In [243]:
x.columns = x.columns.astype(str) 

check Multicollinearity and drop feature has high Multicollinearity with other features

check if there is duplicated in columns

In [244]:
duplicated_columns=final_data.T.duplicated()
print(final_data.columns[duplicated_columns])

Index(['CBlock'], dtype='object')


In [245]:
final_data.drop(['CBlock'],axis=1,inplace=True)

VIP (variance inflation factor)

In [246]:
st=set(final_data.columns)
print(len(st),final_data.shape[1])

106 121


In [247]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

df=final_data.copy()
sz=len(final_data.columns)
cnt=0  # number_of_dropped_features


for i in range(sz):                       # calculate vif sequentially because may be VIF of feature change after drop another feature
    cnt=final_data.shape[1]-df.shape[1]                    
    idx=i-cnt # index of fited column
    vif=variance_inflation_factor(df.values,idx)
    if vif >= 10:
        df.drop(columns=[df.columns[idx]],inplace=True)
        #cnt+=1                           Don't count because there is duplicat in column_names


In [248]:
df.shape

(1452, 93)

measure the significance level of each feature and perform subset selection using the backward-stepwise method

In [249]:
from statsmodels.api import OLS
OLS(y,df).fit().summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Ordinary least squares
================================================================================
Model:                  OLS              Adj. R-squared (uncentered): 0.968     
Dependent Variable:     saleprice        AIC:                         -2697.9011
Date:                   2024-09-14 12:00 BIC:                         -2206.7963
No. Observations:       1452             Log-Likelihood:              1442.0    
Df Model:               93               F-statistic:                 468.2     
Df Residuals:           1359             Prob (F-statistic):          0.00      
R-squared (uncentered): 0.970            Scale:                       0.0085841 
------------------------------------------------------------------------------------
                  Coef.      Std.Err.        t        P>|t|       [0.025      0.975]
------------------------------------------------------------------------------------
bsmtexposure      0.0019       0.0027      0.7006     0.4836     -0.0034      0.0073
bsmtqual         -0.0050       0.0042     -1.1821     0.2374     -0.0133      0.0033
fireplacequ       0.0156       0.0027      5.8551     0.0000      0.0104      0.0208
mszoning          0.0624       0.0053     11.6700     0.0000      0.0519      0.0728
exterqual         0.0031       0.0056      0.5586     0.5765     -0.0079      0.0142
heatingqc        -0.0031       0.0018     -1.6983     0.0897     -0.0067      0.0005
bsmtfintype1      0.0046       0.0019      2.4685     0.0137      0.0009      0.0083
garagefinish     -0.0107       0.0045     -2.3559     0.0186     -0.0196     -0.0018
lotshape          0.0037       0.0021      1.7929     0.0732     -0.0003      0.0077
kitchenqual      -0.0074       0.0042     -1.7637     0.0780     -0.0157      0.0008
BrkTil            0.0090       0.0112      0.8050     0.4210     -0.0129      0.0309
PConc             0.0778       0.0085      9.1504     0.0000      0.0611      0.0945
Slab              0.1622       0.0236      6.8571     0.0000      0.1158      0.2085
Stone             0.0655       0.0399      1.6423     0.1008     -0.0127      0.1437
Wood              0.0713       0.0559      1.2757     0.2023     -0.0383      0.1809
Gambrel           0.0393       0.0299      1.3148     0.1888     -0.0193      0.0980
Hip               0.0180       0.0070      2.5684     0.0103      0.0043      0.0318
Mansard           0.0495       0.0387      1.2786     0.2013     -0.0264      0.1254
Shed             -0.0352       0.0685     -0.5143     0.6071     -0.1695      0.0991
1.5Unf           -0.0393       0.0267     -1.4734     0.1409     -0.0916      0.0130
1Story           -0.0142       0.0098     -1.4477     0.1479     -0.0335      0.0050
2.5Fin            0.1145       0.0358      3.1974     0.0014      0.0442      0.1847
2.5Unf            0.1095       0.0303      3.6152     0.0003      0.0501      0.1689
2Story            0.0637       0.0108      5.9259     0.0000      0.0426      0.0848
SFoyer            0.0149       0.0198      0.7533     0.4514     -0.0239      0.0538
SLvl              0.0259       0.0162      1.6025     0.1093     -0.0058      0.0577
AsphShn           0.0175       0.0716      0.2439     0.8073     -0.1230      0.1579
Brk Cmn           0.0080       0.0650      0.1230     0.9021     -0.1196      0.1356
ImStucc           0.0029       0.0332      0.0863     0.9313     -0.0623      0.0680
Other             0.0833       0.0956      0.8709     0.3839     -0.1043      0.2709
Plywood           0.0070       0.0146      0.4821     0.6298     -0.0216      0.0357
Stone             0.0213       0.0520      0.4093     0.6824     -0.0807      0.1232
Stucco           -0.0193       0.0321     -0.6016     0.5475     -0.0822      0.0436
Wd Sdng           0.0323       0.0159      2.0328     0.0423      0.0011      0.0635
Wd Shng          -0.0275       0.0205     -1.3395     0.1806     -0.0677      0.0128
Stone            

In [250]:
insignificant_features_list=[]
LR_results=OLS(y,df).fit().summary2().tables[1].reset_index().rename(columns={'index':'feature_name'})

while LR_results['P>|t|'].max() > 0.05:
    insignificant_feature=LR_results['feature_name'][ LR_results['P>|t|'] == LR_results['P>|t|'].max() ]
    insignificant_features_list.append(insignificant_feature)
    df.drop(insignificant_feature,axis=1,inplace=True)
    LR_results=OLS(y,df).fit().summary2().tables[1].reset_index().rename(columns={'index':'feature_name'})


In [251]:
insignificant_features_list

[83    mssubclass
 Name: feature_name, dtype: object,
 28    ImStucc
 Name: feature_name, dtype: object,
 27    Brk Cmn
 Name: feature_name, dtype: object,
 26    AsphShn
 Name: feature_name, dtype: object,
 83    yrsold
 Name: feature_name, dtype: object,
 28    Stone
 Name: feature_name, dtype: object,
 26    Plywood
 Name: feature_name, dtype: object,
 42    ClearCr
 Name: feature_name, dtype: object,
 4    exterqual
 Name: feature_name, dtype: object,
 16    Shed
 Name: feature_name, dtype: object,
 60    Basment
 Name: feature_name, dtype: object,
 9    BrkTil
 Name: feature_name, dtype: object,
 23    Stucco
 Name: feature_name, dtype: object,
 0    bsmtexposure
 Name: feature_name, dtype: object,
 19    SFoyer
 Name: feature_name, dtype: object,
 20    Other
 Name: feature_name, dtype: object,
 0    bsmtqual
 Name: feature_name, dtype: object,
 55    Detchd
 Name: feature_name, dtype: object,
 21    NaN
 Name: feature_name, dtype: object,
 25    BrkComm
 Name: feature_name, dtyp

In [252]:
df.shape

(1452, 45)